In [1]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import pickle
import numpy as np
import pandas as pd
import pickle

from multiprocessing import Process
from multiprocessing import Pool
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [2]:

options = Options()

options.add_argument('--user-data-dir=/Users/Arian_Gh/AppData/Local/Google/Chrome/User Data/') 
options.add_argument('--profile-directory=Profile 6')
options.add_argument("--disable-extensions")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--start-maximized")

options.add_argument('disable-infobars')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--remote-debugging-port=9222")
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

prefs = {
    "profile.default_content_setting_values.images": 2
}
options.add_experimental_option("prefs", prefs)

# Add a preference to disable image loading


capabilities = DesiredCapabilities.CHROME.copy()
capabilities['acceptSslCerts'] = True
capabilities['acceptInsecureCerts'] = True




In [3]:
file_path = 'urls.txt'

categories = []

with open(file_path, 'r') as file:
    for line in file:     
        category = line.strip()
        categories.append(category)
file_path = 'cities.txt'

cities = []

with open(file_path, 'r') as file:
    for line in file:     
        cityy = line.strip().lower()
        cities.append(cityy)

In [4]:
all_details = {}
index = 0
urls = []
names = []
for count,category in enumerate(categories):
    driver = webdriver.Chrome(options=options , desired_capabilities=capabilities)
    for city in cities:
        
        try:
            wb_address = category + "?grid_view=1"
            wb_address = wb_address.replace("impcat",city)
            driver.get(wb_address)
            
            time.sleep(2)
            html_text = driver.page_source
            soup = BeautifulSoup(html_text,"lxml")
            
            # If captcha appears
            while "check the box below to proceed" in html_text:
                html_text = driver.page_source
                soup = BeautifulSoup(html_text,"lxml")
                continue

            if "No results found for" in html_text:
                continue


            soup_old = ""
            print(f'Category Number:{count+1}/{len(categories)}----Dict_Length:{len(all_details)}',end='\r')
            while True:


                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                driver.implicitly_wait(3)

                try:

                    btn = driver.find_element(By.CLASS_NAME,"fm2")
                    driver.execute_script("arguments[0].click();",btn)

                except:

                    break
            html_text = driver.page_source
            soup = BeautifulSoup(html_text,"lxml")
            products = soup.find("div",class_="q_hm1 cnhdr fxmn")
            products = products.find("div",class_='lay-left')
            products = products.find("ul",class_="mListGrp w100 sid_df fww wlm mFrgn")
            sections = products.find_all("li")
            filtered_sections = []

            for section in sections:
                if section.has_attr('id'):
                    filtered_sections.append(section)

            for section in filtered_sections:
                details={}
                try:
                    name = section.find("h3").text
                    price = section.find("p").text
                    url = section.find("a")["href"].split(" ")[0]
                except:
                    continue

                if url in urls:
                    continue

                details["name"]= name
                details["price"] = price
                details["url"] = url
                details["category"] = category.split("/")[-1].split(".html")[0]


                all_details[index] = details
                index+=1
        except:
            continue

    driver.close()

Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="recaptcha-anchor"]"}
  (Session info: chrome=116.0.5845.111)
Stacktrace:
	GetHandleVerifier [0x00007FF6A30F52A2+57122]
	(No symbol) [0x00007FF6A306EA92]
	(No symbol) [0x00007FF6A2F3E3AB]
	(No symbol) [0x00007FF6A2F77D3E]
	(No symbol) [0x00007FF6A2F77E2C]
	(No symbol) [0x00007FF6A2FB0B67]
	(No symbol) [0x00007FF6A2F9701F]
	(No symbol) [0x00007FF6A2FAEB82]
	(No symbol) [0x00007FF6A2F96DB3]
	(No symbol) [0x00007FF6A2F6D2B1]
	(No symbol) [0x00007FF6A2F6E494]
	GetHandleVerifier [0x00007FF6A339EF82+2849794]
	GetHandleVerifier [0x00007FF6A33F1D24+3189156]
	GetHandleVerifier [0x00007FF6A33EACAF+3160367]
	GetHandleVerifier [0x00007FF6A3186D06+653702]
	(No symbol) [0x00007FF6A307A208]
	(No symbol) [0x00007FF6A30762C4]
	(No symbol) [0x00007FF6A30763F6]
	(No symbol) [0x00007FF6A30667A3]
	BaseThreadInitThunk [0x00007FFA4E8E7614+20]
	RtlUserThreadStart [0x00007FFA4FA826F1+33]

Message: no such element: Unab

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=116.0.5845.111)
Stacktrace:
	GetHandleVerifier [0x00007FF6A30F52A2+57122]
	(No symbol) [0x00007FF6A306EA92]
	(No symbol) [0x00007FF6A2F3E3AB]
	(No symbol) [0x00007FF6A2F2ABE6]
	(No symbol) [0x00007FF6A2F2B31E]
	(No symbol) [0x00007FF6A2F2B250]
	(No symbol) [0x00007FF6A2F1CC5A]
	(No symbol) [0x00007FF6A2F1D85D]
	(No symbol) [0x00007FF6A2FA0D24]
	(No symbol) [0x00007FF6A2F96FA0]
	(No symbol) [0x00007FF6A2F6D2B1]
	(No symbol) [0x00007FF6A2F6E494]
	GetHandleVerifier [0x00007FF6A339EF82+2849794]
	GetHandleVerifier [0x00007FF6A33F1D24+3189156]
	GetHandleVerifier [0x00007FF6A33EACAF+3160367]
	GetHandleVerifier [0x00007FF6A3186D06+653702]
	(No symbol) [0x00007FF6A307A208]
	(No symbol) [0x00007FF6A30762C4]
	(No symbol) [0x00007FF6A30763F6]
	(No symbol) [0x00007FF6A30667A3]
	BaseThreadInitThunk [0x00007FFA4E8E7614+20]
	RtlU

In [17]:
len(all_details)

0

In [ ]:
    except:
        
        try:
            name = section.find("span",class_="noPdp prd-name").text
            price = section.find("span",class_="elps elps2 fs14 tac clr7").text
            url = None
            details["name"]= name
            details["price"] = price
            details["url"] = url
            details["category"] = category
            all_details[index] = details
            index+=1
        except Exception as e:
            
            continue

In [15]:
df = pd.DataFrame().from_dict(all_details).T
df = df.drop_duplicates()
df = df.reset_index()
#df.to_csv("indiamart.csv")
df

,index,name,price,url,category
0,0,Grinded Green PET Bottle Scrap,₹58/ Kg,https://www.indiamart.com/proddetail/green-pet...,https://dir.indiamart.com/impcat/pet-bottle-sc...
1,1,Multicolor PET Bottle Scrap,₹12/ Kg,https://www.indiamart.com/proddetail/pet-bottl...,https://dir.indiamart.com/impcat/pet-bottle-sc...
2,2,Natural Baled Pet Bottle Scrap,₹51/ Kg,https://www.indiamart.com/proddetail/pet-bottl...,https://dir.indiamart.com/impcat/pet-bottle-sc...
3,3,"Aluminium Bottle Caps Scrap, 25 Kg",₹65/ Kg,https://www.indiamart.com/proddetail/aluminium...,https://dir.indiamart.com/impcat/pet-bottle-sc...
4,4,"Pet Bottle Scrap, Pack Size: Bags",₹30/ Kilogram,https://www.indiamart.com/proddetail/pet-bottl...,https://dir.indiamart.com/impcat/pet-bottle-sc...
...,...,...,...,...,...
1897,2098,Pet Bottle Scrap,,https://www.indiamart.com/wajid-electronic/,https://dir.indiamart.com/impcat/pet-bottle-sc...
1898,2099,Pet Plastic Bottle Scraps,,https://www.indiamart.com/radhe-enteprises-yav...,https://dir.indiamart.com/impcat/pet-bottle-sc...
1899,2100,Plastic Bottle Scrap,₹100/ Kg,https://www.indiamart.com/edmps4709d/,https://dir.indiamart.com/impcat/pet-bottle-sc...
1900,2101,Milky White Hot Washed PET Scrap,₹72/ Kg,https://www.indiamart.com/proddetail/pet-scrap...,https://dir.indiamart.com/impcat/pet-bottle-sc...


In [13]:
df = pd.read_csv("indiamart.csv")
df.head()

,name,price,url,category
0,Inconel 690 Scrap,₹550/ Kilogram,https://www.indiamart.com/proddetail/inconel-6...,Industrial Scrap
1,Titanium Scrap,₹250/ Kilogram,https://www.indiamart.com/proddetail/titanium-...,Industrial Scrap
2,"Stainless Steel Plate Cut Scrap, 50 KG, Packag...",₹100/ Kg,https://www.indiamart.com/proddetail/stainless...,Industrial Scrap
3,"Black Mild Steel Scrap, For Metal Industry, Pa...",₹43/ Kg,https://www.indiamart.com/proddetail/mild-stee...,Industrial Scrap
4,"Aluminium Scrap Lithium Battery, Model Name/Nu...",₹90/ Kg,https://www.indiamart.com/proddetail/scrap-lit...,Industrial Scrap


In [14]:
df["price"] = df["price"].apply(lambda x: str(x).strip(" "))
df["price"] = df["price"].apply(lambda x: str(x).lower())

df["price"] = df["price"].apply(lambda x: str(x).replace("Kilogram","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Kg","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("KG","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Kilogram(s)","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("KILOGRAMS","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kgs","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kilogram","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kg(s)","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("per kg","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kgs","kg"))

df["price"] = df["price"].apply(lambda x: str(x).replace("kg onwards","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Kilogram Onwards","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("TONNE","tonne"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Tonne","tonne"))
df["price"] = df["price"].apply(lambda x: str(x).replace("metric ton(s)","tonne"))
df["price"] = df["price"].apply(lambda x: str(x).replace("ton","tonne") if str(x).split("/")[-1]==" ton" else x)
df["price"] = df["price"].apply(lambda x: str(x).replace("metric ton","tonne"))

df["price"] = df["price"].apply(lambda x: str(x).replace("Ton","tonne"))



In [15]:
df["metric"] = df["price"].apply(lambda x: str(x).split("/")[-1].strip())
df["metric"].value_counts().to_dict()

{'kg': 3215,
 'piece': 365,
 'tonne': 271,
 'unit': 128,
 'pack': 33,
 'set': 27,
 'certificate': 25,
 'plant': 20,
 'project': 17,
 'hour': 16,
 'onwards': 16,
 'number': 13,
 'metric': 13,
 'service': 12,
 'packet': 11,
 'box': 8,
 'nos': 6,
 'day': 6,
 '1 set': 5,
 'dozen': 5,
 'visit': 4,
 'person': 4,
 'meter': 3,
 'piece(s)': 3,
 'month': 3,
 'annum': 3,
 'square meter': 3,
 'qty': 2,
 'pease': 2,
 "20' container": 2,
 'litre': 2,
 'course': 2,
 'certification': 2,
 '₹735': 1,
 '₹606': 1,
 '₹4.50': 1,
 '₹795': 1,
 'gram': 1,
 '₹738': 1,
 'bag': 1,
 'session': 1,
 'starting': 1,
 '1': 1,
 'carton': 1,
 'sq ft': 1,
 '50 gram': 1}

In [19]:
names_to_filter = ["kg", "tonne", "metric"]
filtered_df = df[df['metric'].isin(names_to_filter)]
filtered_df = filtered_df.reset_index(drop=True)

In [21]:
filtered_df.to_csv("filtered_indiamart.csv",index=False)

In [22]:
filtered_df

,name,price,url,category,metric
0,Inconel 690 Scrap,₹550/ kg,https://www.indiamart.com/proddetail/inconel-6...,Industrial Scrap,kg
1,Titanium Scrap,₹250/ kg,https://www.indiamart.com/proddetail/titanium-...,Industrial Scrap,kg
2,"Stainless Steel Plate Cut Scrap, 50 KG, Packag...",₹100/ kg,https://www.indiamart.com/proddetail/stainless...,Industrial Scrap,kg
3,"Black Mild Steel Scrap, For Metal Industry, Pa...",₹43/ kg,https://www.indiamart.com/proddetail/mild-stee...,Industrial Scrap,kg
4,"Aluminium Scrap Lithium Battery, Model Name/Nu...",₹90/ kg,https://www.indiamart.com/proddetail/scrap-lit...,Industrial Scrap,kg
...,...,...,...,...,...
3494,"Dull White Polyester Monofilament Yarn, For Se...",₹165/ kg,https://www.indiamart.com/proddetail/polyester...,Recycled Yarn,kg
3495,Cotton Yarn,₹110/ kg,https://www.indiamart.com/proddetail/cotton-ya...,Recycled Yarn,kg
3496,100% Egyptian Cotton Yarn,₹410/ kg,https://www.indiamart.com/proddetail/100-egypt...,Recycled Yarn,kg
3497,"Dull White Polyester Spun Yarn, For Weaving",₹100/ kg,https://www.indiamart.com/proddetail/polyester...,Recycled Yarn,kg
